In [1]:
import requests
from bs4 import BeautifulSoup as bs
import ujson 
import time
import re
from konlpy.tag import Komoran
from konlpy.tag import Hannanum
import itertools
import mmap
from tqdm import tqdm_notebook
import ujson
import re
import json

In [2]:
# resp = requests.request("GET", "https://www.starfield.co.kr/coexmall/tenant/floorInfo.do")
# soup = bs(resp.text)

headers = {'User-Agent' : 'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.102 Safari/537.36'}
url = 'https://www.starfield.co.kr/coexmall/tenant/categoryInfo.do'
# soup = requests.get(url, headers = headers).text
resp = requests.get(url, headers = headers)
soup = bs(resp.text)

find_ul = soup.find('ul', 'store-list')
find_li = find_ul.find_all('li', 'cube')
for i in find_li :
    k = i.find('span', 'brand-title-ko ko').text
    print(k)
    break
    

C:\Users\dayoung\GroupProject\ArkChatBot\chat_env\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file c:\users\dayoung\anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


#1


In [8]:
url = 'https://www.starfield.co.kr/coexmall/tenant/ajaxSearchCategoryInfo.do'
# base_url = 'https://www.starfield.co.kr/coexmall/tenant/floorInfo.do'
params = {
    'pageCount':16
}
# resp = requests.post(url, params=params)
# resp

In [11]:
# soup = bs(resp.content)
# soup

resp = requests.get(url, params = params)
soup = bs(resp.text, 'lxml')
soup

<html><body><p>{"totalCount":325,"tntData":[{"bcnCd":"","bcnNm":"","bcn_cd":"","busi_tnt_cd":"","cate_detail":[],"cate_list_cn":[],"cate_list_en":[],"cate_list_jp":[],"cate_list_ko":[],"cate_nm_cn":"","cate_nm_en":"","cate_nm_jp":"","cate_nm_ko":"","cate_seq":"","cate_seq_array":[],"coct_end_prid":null,"coct_strt_prid":null,"cord_div":"","cord_seq":"","coupon_list":[],"cp_seq":"","dept_tnt_cd":"","endDate":"","end_hr_min":"22:00","event_list":[],"favorite":false,"firstIndex":0,"fl":"B1","fl_list":[],"floor_id":"","img_logo_uri":"","img_main_bg_logo_uri":"","img_main_bg_mobi_uri":"","img_main_bg_web_uri":"","img_mobi_logo_uri":"/cdn/starfield_02/tnt/20181010/IM20181010011211202540.png","img_thmb_uri":"/cdn/starfield_02/tnt/20181010/IM20181010011211088657.png","insertOrUpdate":"INSERT","irgu_end_hr_min":"","irgu_open_hr_min":"","lastIndex":1,"localeLanguge":"","mama_cate_seq":"","map_id":"","mbrSeq":"","menu_desc":"","menu_list":[],"menu_nm":"","mobi_url":"","modDttm":"","modUsr":"","mod

In [34]:
import requests 
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

## 스타필드 전체 브랜드 크롤링
## 층별, 카테고리별 브랜드 크롤링
## ex) 1층의 레스토랑  = []

base_url = 'https://www.starfield.co.kr/coexmall/tenant/categoryInfo.do'
path = "C:\\Users\\dayoung\\Downloads\\chromedriver_win32\\chromedriver"
driver = webdriver.Chrome(path) ## 경로 설정
driver.implicitly_wait(2)

driver.get(base_url)

html = driver.page_source
soup = bs(html, "html.parser")

category_list = ['//*[@id="CT201802061015172099"]','//*[@id="CT201708291437171739"]', '//*[@id="CT201708291437171738"]', '//*[@id="CT201708291437181740"]',
            '//*[@id="CT201708291437181741"]', '//*[@id="CT201802061015172101"]', '//*[@id="CT201802061015172102"]',
            '//*[@id="CT201708291437181743"]', '//*[@id="CT201802061015172103"]', '//*[@id="CT201708291437181744"]']
data = {}
floor_list = ['B1','B2','1F', '2F']
for floor in floor_list :

    category_dict = {}
    for idx ,category in enumerate(category_list) :
        
        html = driver.page_source
        soup = bs(html, "html.parser")
        
        try :
            driver.find_element_by_xpath('//*[@id="cateOneList"]').click()
        except :
            driver.find_element_by_xpath('//*[@id="cateOneList"]').send_keys(Keys.ENTER)
            
        driver.find_element_by_xpath(category).send_keys(Keys.ENTER)
        for i in range(20) :
            try :
                driver.find_element_by_xpath('//*[@id="viewMore"]/button').click()
            except :
                pass

        html = driver.page_source
        soup = bs(html, "html.parser")
        
        find_ul = soup.find('ul', 'store-list')
        find_li = find_ul.find_all('li', 'cube')
        
        brand_list = []
        for i in find_li :
            j = i.find('div', 'brand-info-wrap')
            k = j.find_all('div', 'info-wrap')[2]
            brand_floor = k.find('span', 'ko').text
            
            if brand_floor == floor : ## 맞다면 저장
                brand_name = i.find('span', 'brand-title-ko ko').text
                brand_list.append(brand_name)
            else :
                pass
        
        cate_list = ['레스토랑&카페', '패션의류', '패션잡화', '뷰티', '라이프스타일', '홈퍼니싱', '키즈', '엔터테인먼트', '슈퍼마켓', '서비스']
        category_dict[cate_list[idx]] = brand_list
    
    data[floor] = category_dict

##{
## 1층 : {rest : []}, shop:[]},
## 2층 : {rest : []}, shop:[]},
## 3층 : {rest : []}, shop:[]}
## }

In [76]:
print(data)  

{'B1': {'레스토랑&카페': ['CJ푸드월드', 'KFC', '[임시휴점] 계절밥상', '가득 드림', '경성면옥', '계절밥상 소반', '고궁 전주 비빔밥', '고디바', '고봉삼계탕', '고운님', '공수간', '공차', '광화문미진', '그랜드마오', '금복당', '니뽕내뽕', '대가집', '더 죠이', '더플레이스', '도모다찌', '동경', '드코닝', '디퀸즈', '라에스키모', '르사이공', '맥도날드', '메이', '멘무샤', '명가분식', '명동칼국수', '모박사 부대찌게', '무교낙지', '미스터순두부', '밀밭', '반하는 보쌈&밥상', '방콕9', '배꼽집', '본죽', '불고기브라더스 도시락점방', '빕스마이픽', '빨라쪼 델 프레도', '뽀르게따', '뽀모도로', '뽕스함바그', '사리원', '사보텐', '사이드쇼', '샤이바나', '서울덮밥&김치찜', '셰프의 국수N육개장', '소노야', '소풍', '소프레소', '쇼군', '스무디킹', '스시 한 판', '스위트 스페이스', '스타벅스 리저브 코엑스몰점', '스타벅스 코엑스몰점', '스타벅스 코엑스별마당점', '신안동찜닭', '신의주찹쌀순대', '쌍둥이 분식 (트윈쿡)', '아그라', '아티제', '안동찜닭&해물떡찜', '앤레커피', '앤티 앤스 프레즐', '어머니황태', '에그슬럿', '예가낙지마을', '웰스', '위니비니', '위치스 커피', '이천가든', '이코복스', '자연은 맛있다', '잠바주스', '전주비빔밥', '제일제면소', '직화한상', '짬뽕신', '쭈불앤쭈불', '차이나팩토리 익스프레스', '초계 국수', '카페 마마스', '카페아이엠티', '커피빈', '케르반', '쿠캣마켓', '크리스피 크림 도넛', '클로리스', '탄탄면공방', '투썸플레이스', '파파밸리', '포메인(코엑스몰점)', '포베이 코엑스몰점', '폴바셋', '피에프창', '하동관', '하코야 코엑스몰점', '한스케익', '할리스커피(코엑스몰점)', '홍대 돈부리', '흑백홍', '힘난다']